
# **PRIMERA PARTE**

##IMPORTS - VARIABLES - URL

### En esta primera entrega de "Extracción de datos" utilizaremos la api del New York Times, utilizaremos dos endpoints:
### 1. Nos va devolver las noticias mas populares del mundo
### 2. Nos devolvera por un rango de mes y año todos los articulos publicados mundialmente.



In [221]:
!pip install -q fastparquet
!pip install psycopg[binary]
!pip install sqlalchemy

#importamos librerias
import os
import configparser
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from datetime import datetime, timedelta, date

import requests
import pandas as pd
import sqlalchemy as sa



In [222]:
#VARIABLES
config = configparser.ConfigParser()
config.read("config.ini")
API_KEY = config["keys"]["API_KEY"]
'''
Obtengo la fecha actual y las separo para poder mandarla por parametro a la url
al hacerlo de esta manera se va actualizar automaticamente y evitar pasarlo en duro.
'''
today = datetime.now()

Year = str(today.year)
Month = str(today.month)

#URL
urlBase = 'https://api.nytimes.com/svc/'

urlMostPopular = urlBase+'topstories/v2/world.json?api-key='
urlYearMonth = urlBase + f"archive/v1/{Year}/{Month}.json?api-key="


In [223]:
#RESPUESTAS

#ARTICULOS MAS POPULARES
r_popular_news =  requests.get(urlMostPopular+API_KEY).json()

#HISTORICO DE ARTICULOS POR MES
r_nyt_month =  requests.get(urlYearMonth+API_KEY).json()



## **FUNCION PARA DEVOLVER LAS NOTICIAS MAS POPULARES**


### Una vez analizado la respuesta JSON creamos una funcion que en este caso llamamos getPopularNews, esta funcion consta de varios pasos:
### 1ero: Creamos un DataFrame con las columnas que vamos a utilizar.
### 2do: con un FOR levantamos la informacion que creemos importante y la **guardamos** en una variable. (Corregimos valores en este caso fechas)
### 3ero: Unimos la informacion de las variables y unimos con las columnas, al final retornamos el dataframe ya listo.

In [224]:
def getPopularNews(r_popular_news):

  #Creacion del DataFrame con las columnas
  df_PopularNews  = pd.DataFrame(columns=['section', 'title', 'abstract', 'byline', 'item_type',
                                          'created_date', 'published_date', 'updated_date', 'link'])

  #Levantar con un FOR la informacion
  for note in r_popular_news['results']:
    #Esto podria ingresar en la parte de limpieza ya que existen 'tipos multimedia' que son videos shorts como youtube y preferi sacarlo ahora
    if note['item_type'] == 'Article':
      section = note['section']
      title = note['title']
      abstract = note['abstract']
      link = note['url']
      byline = note['section']
      item_type = note['item_type']
      created_date = note['created_date']
      published_date = note['published_date']
      updated_date = note['updated_date']

      #Correcion de Fechas
      created_date = str(created_date).split("T")[0]
      published_date = str(published_date).split("T")[0]
      updated_date = str(updated_date).split("T")[0]

      #Creamos un DataFrame con los datos de la nota
      df_note = pd.DataFrame({'section': section,
                               'title': title,
                               'abstract': abstract,
                               'byline': byline,
                               'item_type': item_type,
                               'created_date': created_date,
                               'published_date': published_date,
                               'updated_date': updated_date,
                               'link': link}, index=[0])

      #Concatenamos el DataFrame con el DataFrame original
      df_PopularNews = pd.concat([df_PopularNews, df_note], ignore_index=True)

  return df_PopularNews


In [225]:
dfPopularNews  = getPopularNews(r_popular_news)
dfPopularNews


,section,title,abstract,byline,item_type,created_date,published_date,updated_date,link
0,world,Ukrainians in Germany Weigh Wrenching Choice: ...,"As refugees, they were welcomed with safety, s...",world,Article,2023-12-03,2023-12-03,2023-12-03,https://www.nytimes.com/2023/12/03/world/europ...
1,world,"Maduro, Under Pressure, Holds Vote to Annex Te...",The Venezuelan president is holding a referend...,world,Article,2023-12-03,2023-12-03,2023-12-03,https://www.nytimes.com/2023/12/03/world/ameri...
2,world,Israeli Mothers Knew Their Sons Would Serve. B...,Many mothers are grappling with anxiety as the...,world,Article,2023-12-03,2023-12-03,2023-12-03,https://www.nytimes.com/2023/12/03/world/middl...
3,world,Explosion at Catholic Mass in Philippines Kill...,"The blast, which was claimed by the Islamic St...",world,Article,2023-12-02,2023-12-02,2023-12-03,https://www.nytimes.com/2023/12/02/world/asia/...
4,world,Deadly Paris Knife Attack Revives Terrorism Co...,The authorities said the suspect had serious p...,world,Article,2023-12-02,2023-12-02,2023-12-03,https://www.nytimes.com/2023/12/02/world/europ...
5,world,Tracing the Deep Roots of Ireland’s Support fo...,In a country with its own history of a seeming...,world,Article,2023-12-02,2023-12-02,2023-12-03,https://www.nytimes.com/2023/12/02/world/europ...
6,world,Intelligence Errors Prompt Scrutiny of Israeli...,The Israeli media and The New York Times publi...,world,Article,2023-12-02,2023-12-02,2023-12-03,https://www.nytimes.com/2023/12/02/world/middl...
7,world,Unusual Names Can Complicate Life in Japan. No...,"As such names have increased, so has media att...",world,Article,2023-12-01,2023-12-01,2023-12-02,https://www.nytimes.com/2023/12/01/world/asia/...
8,world,"After Watching 10 Migrants Die at Sea, He Now ...","Witness to a tragedy on a boat to Spain, Moust...",world,Article,2023-12-02,2023-12-02,2023-12-03,https://www.nytimes.com/2023/12/02/world/afric...
9,world,Ex-Chilean Army Officer Is Expelled From U.S. ...,"Pedro Barrientos, 74, was charged in the kidna...",world,Article,2023-12-02,2023-12-02,2023-12-03,https://www.nytimes.com/2023/12/02/world/ameri...


## **FUNCION PARA DEVOLVER UN HISTORICO DE NOTICIAS ACTUALIZADAS POR MES Y AÑO**


In [226]:
# FUNCION DE ARTICULOS HISTORICOS POR MES

def getHistoryNews(r_nyt_month):

  df_articles_month = pd.DataFrame(columns=['section', 'title', 'abstract', 'lead_paragraph', 'byline', 'document_type',
                                          'pub_date', 'total_word_count', 'web_url', 'year', 'month'])
  for note in r_nyt_month['response']['docs']:
    #Esto podria ingresar en la parte de limpieza ya que existen 'tipos multimedia' que son videos shorts como youtube y preferi sacarlo ahora
    if note['document_type'] == 'article':

      headline = note['headline']['main']
      abstrsact =  note['abstract']
      web_url = note['web_url']
      lead_paragraph = r_nyt_month['response']['docs'][0]['lead_paragraph']
      byline =  note['byline']['original']
      document_type = note['document_type']
      word_count =  note['word_count']
      pub_date =  note['pub_date']
      section_name = note['section_name']

      pub_date = str(pub_date).split("T")[0]

      # Creamos un DataFrame con los datos de la nota
      df_note = pd.DataFrame({'section': section_name,
                               'title': headline,
                               'abstract': abstrsact,
                               'lead_paragraph': lead_paragraph,
                               'byline': byline,
                               'document_type': document_type,
                               'pub_date': pub_date,
                               'total_word_count': word_count,
                               'web_url' : web_url}, index=[0])

      #Concatenamos el DataFrame con el DataFrame original
      df_articles_month = pd.concat([df_articles_month, df_note], ignore_index=True)

  # Defino la variable df
  df = df_articles_month

  # Convierto la columna pub_date al tipo datetimelike porque antes era str
  df['pub_date'] = pd.to_datetime(df['pub_date'], format='%Y-%m-%d')


  # Creo las nuevas columnas
  df['year'] = df['pub_date'].dt.year
  df['month'] = df['pub_date'].dt.month
  #Transformo fechas
  df["year"] = pd.to_datetime(df["year"], format="%Y")
  df["year"] = df["year"].dt.strftime("%Y")
  df["pub_date"] = df["pub_date"].dt.strftime("%Y-%m-%d")
  #df["year"] = datetime.datetime.strptime(df['year'], '%Y')










  df = df_articles_month

  return df_articles_month


In [227]:
df_articles_month = getHistoryNews(r_nyt_month)
df_articles_month

,section,title,abstract,lead_paragraph,byline,document_type,pub_date,total_word_count,web_url,year,month
0,Opinion,Donald Trump Still Wants to Kill Obamacare. Why?,"Is this really about policy, or is it personal?",Donald Trump hasn’t talked much about policy i...,By Paul Krugman,article,2023-12-01,880,https://www.nytimes.com/2023/11/30/opinion/tru...,2023,12
1,U.S.,6 Former Jail Officers Charged in Death of Wes...,Prosecutors said the former corrections office...,Donald Trump hasn’t talked much about policy i...,By Jesus Jiménez,article,2023-12-01,673,https://www.nytimes.com/2023/11/30/us/west-vir...,2023,12
2,World,Israel Knew Hamas’s Attack Plan More Than a Ye...,A blueprint reviewed by The Times laid out the...,Donald Trump hasn’t talked much about policy i...,By Ronen Bergman and Adam Goldman,article,2023-12-01,1582,https://www.nytimes.com/2023/11/30/world/middl...,2023,12
3,Business Day,Judge Halts TikTok Ban in Montana,"TikTok, which is owned by the Chinese company ...",Donald Trump hasn’t talked much about policy i...,By Sapna Maheshwari,article,2023-12-01,1030,https://www.nytimes.com/2023/11/30/business/ti...,2023,12
4,U.S.,Kissinger Had the Ear of Presidents. He Had Th...,"In his decades in politics, the statesman advi...",Donald Trump hasn’t talked much about policy i...,By Erica L. Green and Kitty Bennett,article,2023-12-01,962,https://www.nytimes.com/2023/11/30/us/politics...,2023,12
...,...,...,...,...,...,...,...,...,...,...,...
313,Climate,"The Climate Summit Embraces A.I., With Reserva...",The idea of using artificial intelligence to f...,Donald Trump hasn’t talked much about policy i...,By Jim Tankersley,article,2023-12-03,855,https://www.nytimes.com/2023/12/03/climate/art...,2023,12
314,New York,"5 Dead, Including Suspect, and 2 Officers Woun...",The attack occurred early Sunday morning at a ...,Donald Trump hasn’t talked much about policy i...,By Emma Bubola,article,2023-12-03,357,https://www.nytimes.com/2023/12/03/nyregion/st...,2023,12
315,Food,This Chicken Should Be in Every Pot,"Warm spices, sweet root vegetables, savory chi...",Donald Trump hasn’t talked much about policy i...,By Sam Sifton,article,2023-12-03,739,https://www.nytimes.com/2023/12/03/dining/this...,2023,12
316,Opinion,Sharp Views on Dating and Marriage Today,Readers discuss a guest essay about why many w...,Donald Trump hasn’t talked much about policy i...,,article,2023-12-03,1247,https://www.nytimes.com/2023/12/03/opinion/dat...,2023,12


# **SEGUNDA PARTE ALMACENAMIENTO**

### Esta primera particion la voy a realizar sobre el DF de las noticias mas populares, de tal manera que me queden separadas por categoria del articulo ya sea noticias de clima, deporte, mundo etc.

In [228]:

dfPopularNews.to_parquet(
    "datalake/bronze/popular_news_db/popular_news_by_line",
    partition_cols=["byline"],
    engine="fastparquet",
    )

### La segunda paraticion la voy a realizar sobre el DF de todas las noticias del New york times, de una manera mensual y anual, para en caso de ser necesario poder revisar las noticias un mes y año especifico.

### Decidi particionarlo de esta manera ya que si lo automatizamos y se corre el script diariamente no tendriamos duplicados y podriamos generar una base con dividida por mes y año.

In [229]:

df_articles_month.to_parquet(
    "datalake/bronze/all_news_db/all_news_by_date",
    partition_cols=["year", "month"],
    engine="fastparquet",
    )

# **TERCER PARTE**

### En la anterior entrega ya habia realizado transformacion de datos ejemplo: transformacion de fechas, agregado de columnas mes y año separadas para poder particionarlo de una mejor manera y a la vez me sirve en un futuro para hacer consultas SQL y filtrar por esa columnas

### Ahora vamos a revisar que ninguno de los DF contenga duplicados y algunas transformaciones de fechas antes de enviarlas a la base


In [230]:
#TRANSFORMACIONES DE FECHAS
dfPopularNews['created_date'] = pd.to_datetime(dfPopularNews['created_date'])
dfPopularNews['published_date'] = pd.to_datetime(dfPopularNews['published_date'])
dfPopularNews['updated_date'] = pd.to_datetime(dfPopularNews['updated_date'])

#cambie este formato a object ya que para enviarlo a la base de datos es mas facil y al ser solo el año se puede manipular mejor.
df_articles_month['year'] = df_articles_month['year'].astype('object')

In [231]:
#Se podria hacer en una linea directamente pero cree esta funcion para en un futuro caso si hay mas de un df solamente pasarle el nombre
#Lo que hace basicamente primero es averiguar con duplicated() si existe alguno y luego te retorna el index del duplicado

def find_duplicates(df):
    duplicated_rows = df.duplicated()
    return duplicated_rows.index[duplicated_rows]


In [232]:
df_duplicated_rows = find_duplicates(df_articles_month)
df_duplicated_rows

Int64Index([], dtype='int64')

In [233]:
df_duplicated_rows = find_duplicates(dfPopularNews)
df_duplicated_rows

Int64Index([], dtype='int64')

### Me conecto a la base de datos


In [234]:

# Leemos archivo ini
config = configparser.ConfigParser()
config.read('config.ini')

# Extract connection parameters
section = 'postgres'
host = config[section]['host']
port = config[section]['port']
db = config[section]['db']
user = config[section]['user']
pwd = config[section]['pwd']



def connect_to_postgres_db(config_file_path="config.ini", section="postgres"):
 global engine
 global connection
 conn_string = f"postgresql://{user}:{pwd}@{host}:{port}/{db}?sslmode-require"
 engine = create_engine(conn_string)
 connection = engine.connect()

connect_to_postgres_db(config_file_path="config.ini", section="postgres")


## Cree las tablas de una manera que me parecio interesante ya que con el if puede saber si existen o no dependiendo el mensaje de respuesta. Se podria mejorar en una funcion. Pero para tenerl claro visualmente ahora me parece correcto y funcional.

In [235]:
#CREAMOS TABLA ARTICULOS POR MES

# Chequeamos si la tabla existe
if not sa.inspect(engine).has_table("LucianoVenialgo_ArticulosPorMes"):
    # si la tabla no existe, la creamos
    metadata = sa.MetaData()
    table = sa.Table(
        "LucianoVenialgo_ArticulosPorMes",
        metadata,
        sa.Column("section", sa.VARCHAR(200)),
        sa.Column("title", sa.VARCHAR(200)),
        sa.Column("abstract", sa.VARCHAR(1000)),
        sa.Column("lead_paragraph", sa.VARCHAR(1000)),
        sa.Column("byline", sa.VARCHAR(200)),
        sa.Column("document_type", sa.VARCHAR(200)),
        sa.Column("pub_date", sa.VARCHAR(200)),
        sa.Column("total_word_count", sa.Integer),
        sa.Column("web_url", sa.VARCHAR(200)),
        sa.Column("year", sa.VARCHAR(200)),
        sa.Column("month", sa.Integer),
    )

    table.create(engine)
    print("Table 'LucianoVenialgo_ArticulosPorMes' created successfully.")
else:
    # Si ya existe devolvemos este mensaje
    print("Table 'LucianoVenialgo_ArticulosPorMes' already exists.")

Table 'LucianoVenialgo_ArticulosPorMes' created successfully.


In [236]:
#CREAMOS TABLA ARTICULOS MAS POPULARES

# Chequeamos si la tabla existe
if not sa.inspect(engine).has_table("LucianoVenialgo_ArticulosPopulares"):
    # si la tabla no existe, la creamos
    metadata = sa.MetaData()
    table = sa.Table(
        "LucianoVenialgo_ArticulosPopulares",
        metadata,
        sa.Column("section", sa.VARCHAR(200)),
        sa.Column("title", sa.VARCHAR(200)),
        sa.Column("abstract", sa.VARCHAR(1000)),
        sa.Column("byline", sa.VARCHAR(200)),
        sa.Column("item_type", sa.VARCHAR(200)),
        sa.Column("created_date", sa.DATE),
        sa.Column("published_date", sa.DATE),
        sa.Column("updated_date", sa.DATE),
        sa.Column("link", sa.VARCHAR(200)),
    )

    table.create(engine)
    print("Table 'LucianoVenialgo_ArticulosPopulares' created successfully.")
else:
    # Si la tabla existe devolvemos este mensaje
    print("Table 'LucianoVenialgo_ArticulosPopulares' already exists.")

Table 'LucianoVenialgo_ArticulosPopulares' created successfully.


In [237]:
#Enviamos el df a la base de postgres
df_articles_month.to_sql(
    name="LucianoVenialgo_ArticulosPorMes",
    con=engine,
    if_exists="append",
    index=False,
    method='multi',

)

318

In [238]:
dfPopularNews.to_sql(
    name="LucianoVenialgo_ArticulosPopulares",
    con=engine,
    if_exists="append",
    index=False,
    method='multi',

)

36